# Module

- Sequential
- ModuleList
- ModuleDict

In [1]:
import torch
from torch import nn

input_num, hidden_num, output_num = 784, 256, 10


class MLP(nn.Module):
    def __init__(self):
        # 对父类进行初始化方法
        super(MLP, self).__init__()
        self.hidden = nn.Linear(input_num, hidden_num)
        # 隐藏层的激活函数
        self.act = nn.ReLU()
        self.output = nn.Linear(hidden_num, output_num)
        
    # 向前传播算法
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

In [2]:
net = MLP()

In [3]:
net

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)

## Sequential

In [4]:
class MySequential(nn.Module):
    from collections import OrderedDict
    def __init__(self, *args):
        super(MySequential, self).__init__()
        if(len(args) == 1 and isinstance(args[0], OrderedDict)):
            for key, module in args[0].items():
                self.add_module(key, module)
        else:
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)
    
    def forward(self, input):
        for module in self._modules.values():
            input = module(input)
        return input

In [5]:
from collections import OrderedDict
od = OrderedDict([
    ("hidden", nn.Linear(input_num, hidden_num)),
    ("act", nn.ReLU()),
    ("output", nn.Linear(hidden_num, output_num))
])
net = MySequential(od)

In [6]:
net

MySequential(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [7]:
net = MySequential(nn.Linear(input_num, hidden_num),
                  nn.ReLU(),
                  nn.Linear(hidden_num, output_num))
net

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)

## ModuleDict

In [9]:
# 初始化
net = nn.ModuleDict({
    'linear': nn.Linear(input_num, hidden_num),
    'act': nn.ReLU()
})
# 添加元素
net['output'] = nn.Linear(hidden_num, output_num)

In [10]:
net

ModuleDict(
  (act): ReLU()
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [11]:
net.output

Linear(in_features=256, out_features=10, bias=True)

## ModuleList

In [12]:
# 初始化
net = nn.ModuleList([nn.Linear(input_num, hidden_num), nn.ReLU()])
# 添加元素
net.append(nn.Linear(hidden_num, output_num))

ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)

In [15]:
net[2]

Linear(in_features=256, out_features=10, bias=True)

# Paramter

- Parameters/named_parameters
- ParameterList
- ParameterDict

In [1]:
import torch
from torch import nn
from torch.nn import init

In [2]:
net = nn.Sequential(nn.Linear(4, 3),
                   nn.ReLU(),
                   nn.Linear(3, 1))
X = torch.rand(2, 4)
Y = net(X).sum()

In [3]:
# with layer indices
for name, param in net.named_parameters():
    print(name, param.size())

0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [4]:
# within one layer(without indices)
for name, param in net[0].named_parameters():
    print(name, param.shape)

weight torch.Size([3, 4])
bias torch.Size([3])


In [16]:
class MyDictDense(nn.Module):
    def __init__(self):
        super(MyDictDense, self).__init__()
        self.params = nn.ParameterDict({
            'linear1': nn.Parameter(torch.randn(4, 4)),
            'linear2': nn.Parameter(torch.randn(4, 1))
        })
        self.params.update({
            'linear3': nn.Parameter(torch.randn(4, 2))
        })
    
    def forward(self, x, choice = "linear1"):
        return torch.mm(x, self.params[choice])

In [17]:
net = MyDictDense()
x = torch.ones(1, 4)
net(x, 'linear1')

tensor([[-0.4956, -1.4962,  2.6509, -2.5293]], grad_fn=<MmBackward>)